# QFIE for Inverse Pendulum Control

This notebook aims to illustrate how QFIE can control efficiently an inverse pendulum system. The system configuration is reported in the related paper 'On the Implementation of Fuzzy Inference Engines on Quantum Computers - G. Acampora, R. Schiattarella and A.Vitiello', submitted to IEEE Transaction on Fuzzy Systems.

___

Firstly, let us import the required dependencies and let us start by defining the universe of discourse and fuzzy partition for the two input variables $\theta$ and $\omega$ and for the output variable $I$, both for the system controlled by QFIE and the system controlled by a Classical Fuzzy Inference Engine (CFIE).


In [ ]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
import QFIE
import systems
import controllers

You can see the fuzzy partition of the variables of the system by running the following cell. 
The rule base of the system consists in the follwing set:
   - if theta is zero and omega is zero then current is zero;
   - if theta is zero and omega is neg then current is pos_small;
   - if theta is zero and omega is pos then current is neg_small;
   - if theta is pos and omega is zero then current is neg_small;
   - if theta is pos and omega is pos then current is neg_medium;
   - if theta is pos and omega is neg then current is zero;
   - if theta is neg and omega is zero then current is pos_small;
   - if theta is neg and omega is pos then current is zero;
   - if theta is neg and omega is neg then current is pos_medium.
   
Please, refer to the paper for more details about the normalization factors for $\theta$ and $\omega$

In [ ]:
qfie = systems.q_pendulum()
cfie = systems.c_pendulum()

According to the workflow presented in the experimental section of the paper, it will be hereafter shown the case study shown in sec. V-B: there a single execution of QFIE is carried out by starting with a system in the following configuration:
   - $\theta$ = -0.5
   - $\omega$ = 0.5
   - g = 0

Please, execute the following cell to set the system in that configuration.

In [ ]:
T = 1
t = 0.05
theta_0 = -0.5
omega_0 = 0.5
g = False

By running the following cell you can see the action of QFIE on the system. It will be displayed in order: the input values of $\theta$ and $\omega$; the fuzzified value of the aforementioned input variables; the output counts of the quantum circuit after $N_S=8000$ shots; the computed value of current that is applied to the system according to eq. (50) in the paper; the new values of $\theta$ and $\omega$ after a time step of $t=0.05 s$.

Moreover it will be show the quantum circuit implementing QFIE and the histogram containing the probabilities $P_{c_i}$ esteemed(see the paper for thei meaning). 

In [ ]:
q_ctrl = controllers.Quantum_Control(theta_0, omega_0, T, t, g, qfie)
q_ctrl.update(plot_histo=True, draw_qc=True)

___________

Successively, it is presented the comparison of QFIE and CFIE discussed in the final part of the paper. In the following cell you can set the initial configuration of your system, by selecting: 
- T: number of calls to QFIE and CFIE
- t: time step 
- theta_0: initial angle of the pendulum
- omega_0: initial radial velocity of the pendulum

- g: if True gravity is considered, while if False g = 0

In [ ]:
T = 600
t = 0.01
theta_0 = -2
omega_0 = 0

g = False

Then, running the following cell it can be simulated the pendulum controlled by QFIE in the set environment. In the output you can monitor the status of the simulation, call by call to QFIE. As for the previous case, for each call you will see:
   - The crisp value of the input variables
   - The fuzzified values used as input for QFIE
   - The Output Counts of QFIE
   - The computed value of $I$ 
   - The updated crisp values of $\theta$ and $\omega$

Please, note that if T is an high value the whole simulation can take some time.

In [ ]:
q_ctrl = controllers.Quantum_Control(theta_0, omega_0, T, t, g, qfie)
points = q_ctrl.update(plot_histo=False, draw_qc=False)
theta_quantum, omega_quantum = [], []
for p in points:
    theta_quantum.append(p[0])
    omega_quantum.append(p[1])

At this point, executing the following cell it can be simulate the classical control of the system intially in the same condition of that set for the previous case.
You can monitor the simulation at each execution of the classical controll. Indeed, it will be displyed for each call to CIFE:
- The computed value of $I$ 
- The updated crisp values of $\theta$ and $\omega$

In [ ]:
c_ctrl = controllers.Classical_Control(theta_0, omega_0, T, t, g, cfie)
points_c = c_ctrl.update()
theta_c, omega_c = [], []
for p in points_c:
    theta_c.append(p[0])
    omega_c.append(p[1])

Finally, running the following cell it is possible to plot the trends of $\theta$ and $\omega$ controlled by QFIE compared to the ones controlled by CFIE. 

In [ ]:
import matplotlib.pyplot as plt

fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(8, 10))

ax0.plot(theta_c, color='b', label=r'$\theta_c$')
ax0.plot(theta_quantum, color='g', label=r'$\theta_q$')
ax0.set_ylabel(r'$\theta$')
ax0.set_xlabel(r'$T$')
ax0.legend()

ax1.plot(omega_c, color='b', label=r'$\omega_c$')
ax1.plot(omega_quantum, color='g', label=r'$\omega_q$')
ax1.set_ylabel(r'$\omega$')
ax1.set_xlabel(r'$T$')
ax1.legend()



plt.show()





Moreover, the following animation shows the pendulum controlled by the classical and the quantum fuzzy inference engine over the time set. The <b> Red </b> pendulum is controlled by CFIE, the <b> Black</b> one by QFIE. 

In [ ]:
%matplotlib notebook
import math
from matplotlib import animation
fig = plt.figure(figsize=(5, 5), facecolor='w')
ax = fig.add_subplot(1, 1, 1)
plt.rcParams['font.size'] = 15
t_i=[i for i in range(T)]
lns = []
for i in range(T):
    ln, = ax.plot([0, math.sin(theta_quantum[i])], [0, math.cos(theta_quantum[i])],
                  color='k', lw=2, label='Quantum')
    ln_c, = ax.plot([0, math.sin(theta_c[i])], [0, math.cos(theta_c[i])],
                  color='r', lw=2, label='Classical')
    tm = ax.text(-1, 0.9, 'time = %.01f s' % (t_i[i]*t))
    lns.append([ln, ln_c, tm])


ax.set_aspect('equal', 'datalim')
ax.grid()
ani = animation.ArtistAnimation(fig, lns, interval=50)

plt.rcParams['animation.html'] = 'html5'
ani